<a href="https://colab.research.google.com/github/nafamubarokhusni/MALT1-CDD-Tugas-Besar-Bioinformatika/blob/main/CDD_ML_Part_3_MALT1_Descriptor_Dataset_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bioinformatics Project - Computational Drug Discovery [Part 3] Descriptor Calculation and Dataset Preparation**

Pada **Bagian 3**, kita akan menghitung deskriptor molekuler yang pada dasarnya merupakan deskripsi kuantitatif senyawa dalam kumpulan data. Terakhir, kita akan menyiapkannya menjadi kumpulan data untuk pembuatan model berikutnya di Bagian 4.

---

## **Download PaDEL-Descriptor**

In [ ]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

--2024-12-12 02:56:29--  https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip [following]
--2024-12-12 02:56:29--  https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25768637 (25M) [application/zip]
Saving to: ‘padel.zip’

padel.zip           100%[===================>]  24.57M  --.-KB/s    in 0.07s   

2024-12-12 02:56:29 (346 MB/s) - ‘padel.zip’ saved [25768637/25768637]

--2024-12-12 02:56:29--  https://github

In [ ]:
! unzip padel.zip

Archive:  padel.zip
   creating: PaDEL-Descriptor/
  inflating: __MACOSX/._PaDEL-Descriptor  
  inflating: PaDEL-Descriptor/MACCSFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._MACCSFingerprinter.xml  
  inflating: PaDEL-Descriptor/AtomPairs2DFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._AtomPairs2DFingerprinter.xml  
  inflating: PaDEL-Descriptor/EStateFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._EStateFingerprinter.xml  
  inflating: PaDEL-Descriptor/Fingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._Fingerprinter.xml  
  inflating: PaDEL-Descriptor/.DS_Store  
  inflating: __MACOSX/PaDEL-Descriptor/._.DS_Store  
   creating: PaDEL-Descriptor/license/
  inflating: __MACOSX/PaDEL-Descriptor/._license  
  inflating: PaDEL-Descriptor/KlekotaRothFingerprintCount.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._KlekotaRothFingerprintCount.xml  
  inflating: PaDEL-Descriptor/config  
  inflating: __MACOSX/PaDEL-Descriptor/._config  
  inf

## **Load bioactivity data**

Unduh data bioaktivitas ChEMBL yang telah dikurasi dan telah diproses sebelumnya dari Bagian 1 dan 2 dari rangkaian Proyek Bioinformatika ini. Di sini kita akan menggunakan berkas **MALT1_04_bioactivity_data_3class_pIC50.csv** yang pada dasarnya berisi nilai pIC50 yang akan kita gunakan untuk membangun model regresi.

In [ ]:
! wget https://raw.githubusercontent.com/nafamubarokhusni/MALT1-Classification-Tugas-Besar-Bioinformatika/main/2.%20EDA/results/MALT1_04_bioactivity_data_3class_pIC50.csv

--2024-12-12 02:57:29--  https://raw.githubusercontent.com/nafamubarokhusni/MALT1-Classification-Tugas-Besar-Bioinformatika/main/2.%20EDA/results/MALT1_04_bioactivity_data_3class_pIC50.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52216 (51K) [text/plain]
Saving to: ‘MALT1_04_bioactivity_data_3class_pIC50.csv’

MALT1_04_bioactivit 100%[===================>]  50.99K  --.-KB/s    in 0.002s  

2024-12-12 02:57:29 (23.4 MB/s) - ‘MALT1_04_bioactivity_data_3class_pIC50.csv’ saved [52216/52216]



In [ ]:
import pandas as pd

In [ ]:
df3 = pd.read_csv('MALT1_04_bioactivity_data_3class_pIC50.csv')

In [ ]:
df3

,Unnamed: 0,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,0,CHEMBL3632807,CC1(C)CCC2=C(O1)c1ccc([N+](=O)[O-])cc1C(=O)C2=O,active,287.271,2.66030,0.0,5.0,6.397940
1,1,CHEMBL3632806,CC1(C)CCC2=C(O1)c1ccc(C#N)cc1C(=O)C2=O,intermediate,267.284,2.62378,0.0,4.0,5.795880
2,2,CHEMBL3632805,CC1(C)CCC2=C(O1)c1ccc(Br)cc1C(=O)C2=O,intermediate,321.170,3.51460,0.0,3.0,5.036212
3,3,CHEMBL3632804,CC1(C)CCC2=C(O1)c1ccc(F)cc1C(=O)C2=O,intermediate,260.264,2.89120,0.0,3.0,5.823909
4,4,CHEMBL3183233,COCCOc1nc(-c2ccc(Cl)c(Cl)c2)n(-c2ccc(NC(=O)CCl...,intermediate,455.729,4.44360,1.0,6.0,5.481486
...,...,...,...,...,...,...,...,...,...
369,369,CHEMBL5267101,N/C(=N\S(=O)(=O)c1ccncc1)NCc1cc(-c2ccc(Cl)cc2)...,active,501.399,4.03430,2.0,5.0,6.250264
370,370,CHEMBL5266044,N/C(=N\S(=O)(=O)c1cccs1)NCc1cc(-c2ccc(Cl)cc2)n...,active,506.440,4.70080,2.0,5.0,6.603801
371,371,CHEMBL5275402,Cn1ccc(S(=O)(=O)/N=C(\N)NCc2cc(-c3ccc(Cl)cc3)n...,active,504.403,3.37280,2.0,6.0,6.292430
372,372,CHEMBL5290312,N/C(=N\S(=O)(=O)c1cccc(F)c1)NCc1cc(-c2ccc(Cl)c...,active,518.401,4.77840,2.0,4.0,7.251812


In [ ]:
selection = ['canonical_smiles','molecule_chembl_id']
df3_selection = df3[selection]
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [ ]:
! cat molecule.smi | head -5

CC1(C)CCC2=C(O1)c1ccc([N+](=O)[O-])cc1C(=O)C2=O	CHEMBL3632807
CC1(C)CCC2=C(O1)c1ccc(C#N)cc1C(=O)C2=O	CHEMBL3632806
CC1(C)CCC2=C(O1)c1ccc(Br)cc1C(=O)C2=O	CHEMBL3632805
CC1(C)CCC2=C(O1)c1ccc(F)cc1C(=O)C2=O	CHEMBL3632804
COCCOc1nc(-c2ccc(Cl)c(Cl)c2)n(-c2ccc(NC(=O)CCl)cc2)n1	CHEMBL3183233


In [ ]:
! cat molecule.smi | wc -l

374


## **Calculate fingerprint descriptors**


### **Calculate PaDEL descriptors**

In [ ]:
! cat padel.sh

java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv


In [ ]:
! bash padel.sh

Processing CHEMBL3632807 in molecule.smi (1/374). 
Processing CHEMBL3632806 in molecule.smi (2/374). 
Processing CHEMBL3632805 in molecule.smi (3/374). Average speed: 3.18 s/mol.
Processing CHEMBL3632804 in molecule.smi (4/374). Average speed: 1.65 s/mol.
Processing CHEMBL3183233 in molecule.smi (5/374). Average speed: 1.52 s/mol.
Processing CHEMBL15192 in molecule.smi (6/374). Average speed: 1.56 s/mol.
Processing CHEMBL3632802 in molecule.smi (8/374). Average speed: 0.97 s/mol.
Processing CHEMBL3632803 in molecule.smi (7/374). Average speed: 1.12 s/mol.
Processing CHEMBL3632801 in molecule.smi (9/374). Average speed: 0.92 s/mol.
Processing CHEMBL3632800 in molecule.smi (10/374). Average speed: 0.81 s/mol.
Processing CHEMBL3632799 in molecule.smi (11/374). Average speed: 0.78 s/mol.
Processing CHEMBL3635003 in molecule.smi (12/374). Average speed: 0.80 s/mol.
Processing CHEMBL3635002 in molecule.smi (13/374). Average speed: 0.74 s/mol.
Processing CHEMBL3635001 in molecule.smi (14/374)

In [ ]:
! ls -l

total 25924
-rw-r--r-- 1 root root   676312 Dec 12 02:59 descriptors_output.csv
drwxr-xr-x 3 root root     4096 Dec 12 02:56 __MACOSX
-rw-r--r-- 1 root root    52216 Dec 12 02:57 MALT1_04_bioactivity_data_3class_pIC50.csv
-rw-r--r-- 1 root root    24396 Dec 12 02:57 molecule.smi
drwxrwxr-x 4 root root     4096 May 30  2020 PaDEL-Descriptor
-rw-r--r-- 1 root root      231 Dec 12 02:56 padel.sh
-rw-r--r-- 1 root root 25768637 Dec 12 02:56 padel.zip
drwxr-xr-x 1 root root     4096 Dec 10 17:46 sample_data


## **Preparing the X and Y Data Matrices**

### **X data matrix**

In [ ]:
df3_X = pd.read_csv('descriptors_output.csv')

In [ ]:
df3_X

,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,CHEMBL3632806,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL3632807,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CHEMBL3632804,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CHEMBL3632805,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL15192,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,CHEMBL5267101,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
370,CHEMBL5266044,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
371,CHEMBL5275402,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372,CHEMBL5290312,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df3_X = df3_X.drop(columns=['Name'])
df3_X

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
370,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
371,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
372,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## **Y variable**

### **Convert IC50 to pIC50**

In [ ]:
df3_Y = df3['pIC50']
df3_Y

,pIC50
0,6.397940
1,5.795880
2,5.036212
3,5.823909
4,5.481486
...,...
369,6.250264
370,6.603801
371,6.292430
372,7.251812


## **Combining X and Y variable**

In [ ]:
dataset3 = pd.concat([df3_X,df3_Y], axis=1)
dataset3

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.397940
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.795880
2,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.036212
3,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.823909
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.481486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.250264
370,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.603801
371,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.292430
372,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.251812


In [ ]:
dataset3.to_csv('MALT1_06_bioactivity_data_3class_pIC50_pubchem_fp.csv', index=False)

# **Mari unduh file CSV ke komputer lokal Anda untuk Bagian 4 (Pembuatan Model).**